In [13]:
from google import genai
from PIL import Image
import os
import re
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
image = Image.open('image.png')

In [15]:
client = genai.Client(api_key=os.getenv("GENAI_API_KEY"))

In [16]:
prompt = (
    "Transcribe the nutrition facts table in this image and output the data "
    "as a single **JSON object**. Use keys like 'serving-size', 'energy-kcal', 'fat', 'carbohydrates', 'proteins', 'saturated-fat', 'trans-fat', 'sugars', 'added-sugars', 'sodium', 'salt', and 'fiber'. "
    "Do not include any text outside of the JSON object."
    "If the values are not in the image, fill the values with 0"
    "Pay attention to the unit, normalize the unit so it's stated in g (gram) instead of mg (miligram)"
)

In [17]:
result = client.models.generate_content(
    model='gemini-2.5-flash', 
    contents=[prompt, image]
)

In [18]:
data = result.text
print(data)

```json
{
  "serving-size": "21g",
  "energy-kcal": 60,
  "fat": "0g",
  "carbohydrates": "17g",
  "proteins": "0g",
  "saturated-fat": "0g",
  "trans-fat": "0g",
  "sugars": "17g",
  "added-sugars": "17g",
  "sodium": "0g",
  "salt": "0g",
  "fiber": "0g"
}
```


In [19]:
cleanedData = data.strip().replace('```json', '').replace('```', '').replace('\n','').strip()
print(cleanedData)

cleanedData = re.sub(r'\s+', ' ', cleanedData).strip()

{  "serving-size": "21g",  "energy-kcal": 60,  "fat": "0g",  "carbohydrates": "17g",  "proteins": "0g",  "saturated-fat": "0g",  "trans-fat": "0g",  "sugars": "17g",  "added-sugars": "17g",  "sodium": "0g",  "salt": "0g",  "fiber": "0g"}


In [20]:
tempDict = {}

for i in cleanedData.split(','):
    x = i.split(':')
    col = (x[0].split('"'))[1]
    num = x[1]

    tempDict[col] = num

    print(i)

{ "serving-size": "21g"
 "energy-kcal": 60
 "fat": "0g"
 "carbohydrates": "17g"
 "proteins": "0g"
 "saturated-fat": "0g"
 "trans-fat": "0g"
 "sugars": "17g"
 "added-sugars": "17g"
 "sodium": "0g"
 "salt": "0g"
 "fiber": "0g"}


In [21]:
resDict = {}

for i in tempDict:
    if i == 'serving-size':
        divider = tempDict[i].split('"')
        divider = divider[1].split('g')
        divider = int(divider[0].strip())
    else:
        try:
            resDict[i+'_1g'] =  round(float(tempDict[i].strip())/divider, 3)
        except ValueError:
            value = tempDict[i].split("}")
            value = float(value[0].strip())

            resDict[i+'_1g'] =  round(value/divider, 3)

resDict

ValueError: could not convert string to float: '"0g"'